# Lesson 2 — Line Following (Decisions + Movement Reuse)

Big idea: the robot can sense — now it must decide.

Rules:
- You **must reuse** movement from Lesson 1 (`moves.py`).
- No new movement code in this lesson.


In [ ]:
# --- Bootstrap (same in every notebook) ---
from pathlib import Path
import sys

def add_repo_root():
    here = Path.cwd().resolve()
    for p in [here] + list(here.parents):
        if (p / 'lessons').is_dir() and (p / 'common').is_dir():
            if str(p) not in sys.path:
                sys.path.insert(0, str(p))
            print('Repo root:', p)
            return p
    raise FileNotFoundError('Could not find repo root (needs lessons/ and common/)')

add_repo_root()

In [ ]:
import time, importlib
from fast_hi_wonder import InfraredSensors
from lesson01.level_1 import moves

importlib.reload(moves)
ir = InfraredSensors()

def read_ir():
    vals = ir.read_sensor_data()
    return tuple(1 if v else 0 for v in vals)

print('✔ IR ready; moves imported')
print('Sensor sample:', read_ir())

## Map your sensors

We need to know which index is Left / Centre / Right.

Run this while you place the robot over the line and move it.


In [ ]:
for _ in range(20):
    print(read_ir())
    time.sleep(0.2)

## Decision loop (starter)

Update indices below if needed.

This is a timed loop (so Jupyter doesn't feel stuck).


In [ ]:
# Adjust these once you know ordering
IDX_LEFT = 0
IDX_MID  = 1
IDX_RIGHT = -1

def step(style='smooth'):
    s = read_ir()
    left = s[IDX_LEFT] == 1
    mid  = s[IDX_MID] == 1 if len(s) > 2 else False
    right= s[IDX_RIGHT] == 1

    # print decisions for learning
    print('IR', s, '->', 'L' if left else '-', 'M' if mid else '-', 'R' if right else '-')

    if style == 'aggressive':
        fwd = 40
        nudge = 32
    else:
        fwd = 30
        nudge = 24

    if mid and not left and not right:
        moves.forward(0.08, speed=fwd)
        return 'straight'
    if left and not right:
        moves.move_left(0.06, speed=nudge)
        return 'left'
    if right and not left:
        moves.move_right(0.06, speed=nudge)
        return 'right'

    # lost
    moves.stop_now()
    return 'lost'

def run(seconds=12, style='smooth'):
    end = time.time() + float(seconds)
    try:
        while time.time() < end:
            step(style=style)
            time.sleep(0.03)
    finally:
        moves.stop_now()

run(seconds=10, style='smooth')

### Challenge ladder
- 🟢 Follow the line
- 🔵 Handle sharper corners
- 🟣 Style mode (aggressive vs smooth)
- 🔴 Explain your decisions out loud
